<a href="https://colab.research.google.com/github/SvetaRakh/Start_v_DS/blob/main/Decorators/Decorators_homework_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание: декораторы

1. Реализуйте декоратор benchmark(func), выводящий время, которое заняло выполнение декорируемой функции

2. Реализуйте декоратор logging(func), который выводит параметры с которыми была вызвана функция

3. Реализуйте декоратор counter(func), считающий и выводящий количество вызовов декорируемой функции

4. Реализуйте декоратор memo(func), запоминающий результаты исполнения функции func, чьи аргументы *args должны быть хешируемыми. Сравните время выполнения рекурсивной реализации расчета чисел Фибоначчи без декоратора и с ним

## Импорт библиотек, установка констант

In [ ]:
import requests
import time
import re

import functools
from functools import wraps

from random import randint

In [ ]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [ ]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        return_value = func(*args, **kwargs)
        end = time.time()
        print(f'Время выполнения функции {func.__name__} составило {end - start:.6f} секунд')
        print()
        return return_value
    return wrapper

## Задание 2

In [ ]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        fname = func.__name__                 # получение имени функции
        result = func(*args, **kwargs)
        print(f'Функция {fname} вызвана с параметрами: \nargs: {args}, kwargs: {kwargs}\n')
        return result

    return wrapper

## Задание 3

In [ ]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal w_cnt
        result = func(*args, **kwargs)
        w_cnt += 1
        print(f'Функция {func.__name__} была вызвана: {w_cnt} раз')
        print()
        return result
    w_cnt = 0
    return wrapper

## Задание 4

In [ ]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}

  def fmemo(*args):
      if args in cache:
         return cache[args]
      result = func(*args)
      cache[args] = result
      return result

  fmemo.cache = cache
  return fmemo

## Тестирование

In [ ]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Время выполнения функции word_count составило 0.809189 секунд

Функция word_count вызвана с параметрами: 
args: ('whole',), kwargs: {}

Функция word_count была вызвана: 1 раз

Cлово whole встречается 176 раз


In [ ]:
# декоратор для рекурсивной функции
def benchmark_recur(func, currently_evaluating=set()):
    """
    Декоратор, выводящий общее время выполнения декорируемой рекурсивной функции
    """
    is_evaluating = False
    def wrapper(*args, **kwargs):
        nonlocal is_evaluating
        if is_evaluating:
            return func(*args, **kwargs)
        else:
            start_time = time.perf_counter()
            is_evaluating = True
            try:
                return_value = func(*args, **kwargs)
            finally:
                is_evaluating = False
            end_time = time.perf_counter()
            diff = end_time - start_time
            print(f'Общее время выполнения функции {func.__name__}{args} составило {end_time - start_time:.6f} секунд')
            return return_value
    return wrapper

In [ ]:
@benchmark_recur
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [ ]:
# измеряем время выполнения
fib(15)

Общее время выполнения функции fib(15,) составило 0.001117 секунд


610

In [ ]:
@benchmark_recur
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [ ]:
# измеряем время выполнения
fib(15)

Общее время выполнения функции fmemo(15,) составило 0.000065 секунд


610